# IF3270 Machine Learning | Tugas Besar 2


Group Number: 46

Group Members:
- Ariel Herfrison (13522002)
- Zachary Samuel Tobing (13522016)
- Imam Hanif Mulyarahman (13522030)

## Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(42)

## Import Dataset

In [ ]:
# train = pd.read_csv("./train.csv")
# test = pd.read_csv("./test.csv")

# Split Training Set and Validation Set

In [ ]:
# Write code here

# Data Cleaning and Preprocessing

In [ ]:
# Write code here

## Compile Preprocessing Pipeline

In [ ]:
# Write code here

# Modeling and Validation

## 4.1 Variation 1

### 4.1.1 Modeling

#### 4.1.1.1 Model 1

In [ ]:
# ...

#### 4.1.1.2 Model 2

In [ ]:
# ...

#### 4.1.2. Comparison and Analysis

In [ ]:
# ...

`Provide your analysis here`

# 5. Model from Scratch

## 5.1. Modeling

In [ ]:
# ...

## 5.2. Comparison & Analysis

In [ ]:
# ...

`Provide your analysis here`